In [9]:
from langchain_experimental.graph_transformers.diffbot import DiffbotGraphTransformer
from config import env
import os

In [2]:
diffbot_nlp = DiffbotGraphTransformer(diffbot_api_key=os.getenv("DIFFBOT_API_KEY"))

In [20]:
from langchain_community.document_loaders import WikipediaLoader

query = "Warren Buffett"
raw_documents = WikipediaLoader(query=query).load()
graph_documents = diffbot_nlp.convert_to_graph_documents(raw_documents)

In [21]:
graph_documents[0]

GraphDocument(nodes=[Node(id='http://www.wikidata.org/entity/Q47213', type='Person', properties={'name': 'Warren Buffett', 'age': '7', 'dateOfBirth': '1930-08-30', 'positionHeld': 'investor'}), Node(id='http://www.wikidata.org/entity/Q203807', type='Organization', properties={'name': 'The Giving Pledge', 'foundingDate': '2010'}), Node(id='Buffett Partnership Ltd.', type='Organization', properties={'name': 'Buffett Partnership Ltd.', 'foundingDate': '1956'}), Node(id='http://www.wikidata.org/entity/Q723488', type='Person', properties={'name': 'Howard Homan Buffett', 'positionHeld': 'congressman'}), Node(id='Leila', type='Person', properties={'name': 'Leila', 'numberOfChildren': '3'}), Node(id='http://www.wikidata.org/entity/Q5284', type='Person', properties={'name': 'Bill Gates'}), Node(id='http://www.wikidata.org/entity/Q8134', type='Skill', properties={'name': 'economics'}), Node(id='http://www.wikidata.org/entity/Q1505660', type='Skill', properties={'name': 'textile manufacturing'}),

In [22]:
from langchain_neo4j import Neo4jGraph

url = "bolt://localhost:7687"
username = "neo4j"
password = "password"

graph = Neo4jGraph(url=url, username=username, password=password)

In [23]:
graph.add_graph_documents(graph_documents, include_source=True)

In [24]:
graph.refresh_schema()

# Query the graph

In [25]:
from langchain_neo4j import GraphCypherQAChain
from langchain_aws import ChatBedrock

# Create the Cypher and QA LLMs

# Cypher LLM for generating Cypher queries
cypher_llm_bedrock_model = ChatBedrock(
    temperature=0,
    model="us.anthropic.claude-opus-4-20250514-v1:0",
    region=env.AWS_REGION,
    aws_access_key_id=env.AWS_ACCESS_KEY_ID,
    aws_secret_access_key=env.AWS_SECRET_ACCESS_KEY,
)

# QA LLM for answering questions based on the Cypher query results
qa_llm_bedrock_model = ChatBedrock(
    temperature=0,
    model="us.anthropic.claude-opus-4-20250514-v1:0",
    region=env.AWS_REGION,
    aws_access_key_id=env.AWS_ACCESS_KEY_ID,
    aws_secret_access_key=env.AWS_SECRET_ACCESS_KEY,
)

chain = GraphCypherQAChain.from_llm(
    cypher_llm=cypher_llm_bedrock_model,
    qa_llm=qa_llm_bedrock_model,
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True,
)

In [ ]:
chain.invoke("Who is or was working at Berkshire Hathaway?")